# Simple SD Webui Launcher

### One-click launch Stable Diffusion Webui for Google Colab. No accounts/tokens required. 

Select options then run the cell.

Cloning webui repository from https://github.com/AUTOMATIC1111/stable-diffusion-webui

Previous version of this notebook: https://colab.research.google.com/github/Miraculix200/StableDiffusionUI_Colab/blob/main/StableDiffusionUI_Colab.ipynb

### Links to models used in settings



https://huggingface.co/Linaqruf/anything-v3.0  
https://huggingface.co/SpiteAnon/gigachad-diffusion  
https://huggingface.co/prompthero/midjourney-v4-diffusion  
https://huggingface.co/nitrosocke/mo-di-diffusion  
https://huggingface.co/TheLastBen/hrrzg-style-768px  
https://huggingface.co/acheong08/f222  
https://huggingface.co/acheong08/secretAI  
https://huggingface.co/hakurei/waifu-diffusion-v1-4  
https://huggingface.co/julienacquaviva/inpainting  
https://huggingface.co/acheong08/SD-V1-5-cloned  

# Launch webui

Run this cell to start the webui

In [ ]:
#@markdown # Mount Google Drive to speed up webui launch
#@markdown Default folders on Google Drive: `embeddings`, `repositories`, `models`
mount_gdrive = True #@param {type:"boolean"}

#@markdown # Install xformers and default settings
install_xformers = False #@param {type:"boolean"}
install_default_settings = True #@param {type:"boolean"}

#@markdown # Install webui extensions
install_image_browser_extension = True #@param {type:"boolean"}
install_tag_editor_extension = False #@param {type:"boolean"}

#@markdown # Choose one or more models to download
#@markdown Set all to `False` if model is already present on Google Drive
model_15 = True #@param {type:"boolean"}
model_inpainting = False #@param {type:"boolean"}
model_anything_v3 = False #@param {type:"boolean"}
#model_anythingXtron = False #@param {type:"boolean"}
model_fred_herzog = False #@param {type:"boolean"}
model_f222 = False #@param {type:"boolean"}
model_secret_ai = False #@param {type:"boolean"}
model_waifu = False #@param {type:"boolean"}
model_gigachad = False #@param {type:"boolean"}
model_midjourney_v4 = False #@param {type:"boolean"}
model_modern_disney_style = False #@param {type:"boolean"}

#@markdown # Run localtunnel or ngrok reverse proxy instead of gradio
use_localtunnel = True #@param {type:"boolean"}
ngrok_token = "" #@param {type:"string"}
ngrok_region = "eu" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"] {type:"raw"}

#@markdown # Additional commandline parameters and launch options
#@markdown See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings
cmd_line_params = "--gradio-debug --disable-console-progressbars" #@param {type:"string"}
webui_port = 9001 #@param {type:"integer"}

#@markdown # Credentials for login to the webui (not necessary)
webui_username = "" #@param {type:"string"}
webui_password = "" #@param {type:"string"}

######################################################

CFG_URL = "https://raw.githubusercontent.com/Miraculix200/StableDiffusionUI_Colab/main/config.json"
CFG_UI_URL = "https://raw.githubusercontent.com/Miraculix200/StableDiffusionUI_Colab/main/ui-config.json"
XFORMERS_URL = "https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl"

# models which don't require a Huggingface token
MODEL_INPAINTING_URL = "https://huggingface.co/julienacquaviva/inpainting/resolve/main/sd-v1-5-inpainting.ckpt"
MODEL_15_URL = "https://huggingface.co/acheong08/SD-V1-5-cloned/resolve/main/v1-5-pruned-emaonly.ckpt"

webui_drive_folder = "/content/drive/MyDrive/SD_webui_Colab/"
webui_folder = "/content/stable-diffusion-webui/"

models_folder = webui_folder + "models/Stable-diffusion/"

######################################################

import os
import time
from google.colab import drive

# list of folder to be linked to gdrive
def linkFoldersGdrive():
  mvToGdrive("embeddings")
  #mvToGdrive("extensions")
  #mvToGdrive("textual_inversion")
  mvToGdrive("repositories")
  mvToGdrive("models")
  #mvToGdrive("outputs")

# function for creating folders on gdrive and symlinks
def mvToGdrive(path_from_webui_root):
  if not os.path.exists(webui_drive_folder + path_from_webui_root):
    print("Creating folder: " + webui_drive_folder + path_from_webui_root)
    !mkdir -p {webui_drive_folder}{path_from_webui_root}

  if os.path.exists(webui_folder + path_from_webui_root):
    print("Copying folder: " + webui_folder + path_from_webui_root)
    !cp -r {webui_folder}{path_from_webui_root} {webui_drive_folder}
    print("Deleting folder: " + webui_folder + path_from_webui_root)
    !rm -rf {webui_folder}{path_from_webui_root}

  !ln -sv {webui_drive_folder}{path_from_webui_root} {webui_folder}{path_from_webui_root}

######################################################

# check GPU availability
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

sys.exit("ohai")

# install xformers
if install_xformers:
  %pip install -q {XFORMERS_URL}
  cmd_line_params = cmd_line_params + " --force-enable-xformers"

# clone webui repo
!git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui {webui_folder}

# download default settings
if install_default_settings:
  %cd /content/stable-diffusion-webui
  !wget {CFG_URL}
  !wget {CFG_UI_URL}

# install image browser extension
if install_image_browser_extension:
  %cd {webui_folder}extensions
  !git clone --depth 1 https://github.com/yfszzx/stable-diffusion-webui-images-browser

# install Dataset Tag Editor extension
if install_tag_editor_extension:
  %cd {webui_folder}extensions
  !git clone --depth 1 https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor.git

# mount google drive
if mount_gdrive:
  drive.mount('/content/drive')
  linkFoldersGdrive()

#  # temporarily copy repositories from gdrive to session filesystem
#  # note sure if necessary, so they can be updated by git
#  repo_path = webui_folder + "repositories"
#  if os.path.islink(repo_path):
#    print("Deleting repositories symlink and copying repositories/ folder to local fs")
#    !rm {webui_folder}repositories
#    !cp -rp {webui_drive_folder}repositories {webui_folder}repositories
#    print("Installing requirements")
#    # install requirements and exit
#    %cd {webui_folder}
#    !COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py
#    mvToGdrive("repositories")

######################################################

# download models
%cd {models_folder}

if model_15:
  !wget {MODEL_15_URL}
  !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5-pruned-emaonly.vae.pt 

if model_inpainting:
  !wget {MODEL_INPAINTING_URL}

if model_anything_v3:
  !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt
  !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -O Anything-V3.0-pruned.vae.pt

#if model_anythingXtron:
#  !wget https://huggingface.co/CrasHthe2nd2/anything-tron/resolve/main/anythingXtron.ckpt
#  !wget https://huggingface.co/CrasHthe2nd2/anything-tron/resolve/main/Anything-V3.0.vae.pt -O anythingXtron.vae.pt

if model_gigachad:
  !wget https://huggingface.co/SpiteAnon/gigachad-diffusion/resolve/main/gigachad_2000.ckpt

if model_midjourney_v4:
  !wget https://huggingface.co/prompthero/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt

if model_modern_disney_style:
  !wget https://huggingface.co/nitrosocke/mo-di-diffusion/resolve/main/moDi-v1-pruned.ckpt

if model_fred_herzog:
  !wget https://huggingface.co/TheLastBen/hrrzg-style-768px/resolve/main/hrrzg.ckpt

if model_f222:
  !wget https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt

if model_secret_ai:
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/animefull-final-pruned/model.ckpt -O secret_ai.ckpt
    %cd {webui_folder}models/VAE/
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/animevae.pt
    !mkdir -p {webui_folder}/models/hypernetworks/
    %cd {webui_folder}models/hypernetworks/
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/aini.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime_2.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime_3.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_2.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_3.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_kemono.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_protogen.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_scalie.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_transformation.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/pony.pt

if model_waifu:
  %cd {models_folder}
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-penultimate-ucg-cont.ckpt
  %cd {webui_folder}models/VAE/
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt

######################################################

# install and launch localtunnel
if ngrok_token != "":
  cmd_line_params = cmd_line_params + " --ngrok " + ngrok_token + " --ngrok-region " + ngrok_region
elif use_localtunnel:
  if not os.path.exists('/tools/node/bin/lt'):
    print("Installing localtunnel")
    !npm install -g localtunnel
  !nohup lt --port 7860 > /content/srv.txt 2>&1 &
  time.sleep(2)
  print("The webui will run on this URL:")
  !grep -o 'https[^ ]*' /content/srv.txt
else:
  cmd_line_params = cmd_line_params + " --share"

######################################################

# launch webui
cmd_line_params = cmd_line_params + " --port " + webui_port

if webui_username != "":
  cmd_line_params = cmd_line_params + " --gradio-auth " + webui_username + ":" + webui_password

!python launch.py {cmd_line_params}

######################################################

# Optional cells

## Zip images for downloading on local drive
After running the cell, click the folder icon on the left, the one below {x}, search for `outputs.zip` in the `/content` folder

In [ ]:
zip_folder = "/content/stable-diffusion-webui/outputs" #@param {type:"string"}
!zip -r /content/outputs.zip {zip_folder}

## Stable Diffusion Pickle Scanner: scan models for malware

Scan .pt, .ckpt and .bin files for potentially malicious code

Source: https://github.com/zxix/stable-diffusion-pickle-scanner



In [ ]:
scan_folder = "/content/stable-diffusion-webui/models" #@param {type:"string"}
%cd /content/stable-diffusion-webui
!wget https://raw.githubusercontent.com/zxix/stable-diffusion-pickle-scanner/main/pickle_scan.py
!wget https://raw.githubusercontent.com/zxix/stable-diffusion-pickle-scanner/main/pickle_inspector.py

!python pickle_scan.py {scan_folder}

## Download a model from Google Drive

Copy the file ID from your url and paste it into the GOOGLE_FILE_ID text field below, before running this cell

In [ ]:
###########################################################
google_file_id = "1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA" #@param {type:"string"}
download_folder = "/content/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
###########################################################

# above file ID is Stable Diffusion model 1.4. Replace it with the file ID of the model you want to download from Google Drive

%cd {download_folder}
url = "https://drive.google.com/u/0/uc?id=" + google_file_id + "&export=download&confirm=t"
!gdown $url

## Download a model from MEGA


Change MEGA_DOWNLOAD_URL to the URL of the model, then run the cell

Download can be very slow (>30 minutes for a 4GB model)

In [ ]:
##########################################################
mega_download_url = "https://mega.nz/file/mdcVARhL#FUq5TL2xp7FuzzgMS4B20sOYYnPZsyPMw93sPMHeQ78" #@param {type:"string"}
download_folder = "/content/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
##########################################################

%cd {download_folder}

%pip install mega.py
from mega import Mega

mega = Mega()
print("Connecting to MEGA")
m = mega.login()
print("Downloading file (no progress display available)")
m.download_url(mega_download_url)
print("Download complete")